### Import modules

In [1]:
import os
import pandas as pd

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA

os.chdir("../")
from src.ragas.ragas_utils import run_evaluation, upload_csv_dataset_to_langsmith



### Load the existing Chroma instance

In [3]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma(persist_directory="chroma", embedding_function=embeddings)

#### Create retriever

In [4]:
# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [5]:
retriever.invoke("What is the article about?")

[Document(page_content='of news stories about a really sensitive issue that\'s been very difficult for either party to solve."'),
 Document(page_content='and I" (DW) This year, the jury chose two winners in the category Online: "Digital journalism has a lot to offer. The two prizewinners represent a different approach in an interesting way and show how journalism generally evolves with multimedia possibilities," explains the jury. In the first contribution, Christian Salewski und Felix Rohrbeck track the disposal of electronic scrap in Germany and find out that it isn\'t always legal and fair. In the second contribution, a group of Deutsche Welle trainees asked their grandmothers from Belarus, Brazil, Chile, China, Kenya and Germany about their'),
 Document(page_content='is, in journalism, if we gather the "facts," we can usually find the answers to what we\'re looking for.  When it comes to God, Jesus and the Holy Spirit, those answers rest in faith. As a journalist, I seek intellectu

In [6]:
# Initialize language model
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

#### Create a RAG chain

In [7]:
rag_chain = RetrievalQA.from_chain_type(
  llm = llm,
  chain_type = "stuff",
  retriever = retriever,
  return_source_documents = True,
)

## Evaluation the RAG system using RAGAS

### Run the evalution

In [14]:
result = run_evaluation(rag_chain)

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

In [15]:
result.to_pandas().head()

,question,contexts,answer,ground_truth,answer_correctness,faithfulness,answer_relevancy,context_precision
0,What upcoming animated project will feature Ad...,[(The Hollywood Reporter)The skies over Gotham...,The upcoming animated project that will featur...,Adam West and Burt Ward will be reprising thei...,0.806599,1.0,0.000000,1.000000
1,What animated project did Adam West and Burt W...,[(The Hollywood Reporter)The skies over Gotham...,Adam West and Burt Ward announced an upcoming ...,Adam West and Burt Ward announced a new animat...,0.842686,1.0,0.000000,1.000000
2,What event is Rory McIlroy preparing for after...,[for 2014-15. He hurt his shoulder in December...,Rory McIlroy is preparing for Arnold Palmer's ...,Rory McIlroy is preparing for the U.S. Masters...,0.233269,0.5,0.863586,0.833333
3,How did Donald Trump help Rory McIlroy retriev...,[(CNN)With a little bit of help from Donald Tr...,"Donald Trump, the owner of the Blue Monster co...",Donald Trump helped Rory McIlroy retrieve his ...,0.613383,1.0,0.893972,1.000000
4,What caused the collapse of the Iraqi army dur...,"[of 2014. ISIS used speed, discipline and ruth...",The collapse of the Iraqi army during the ISIS...,The collapse of the Iraqi army during the ISIS...,0.770911,1.0,0.975391,1.000000


### Upload evaluation set to langsmith


In [23]:
dataset_name = "cnn_dailymail_testset"
dataset_desc = "Synthetic testset data for Huggingface CNN Dailymail dataset."

dataset = uupload_csv_dataset_to_langsmith(dataset_name=dataset_name, dataset_desc=dataset_desc)

Created a new dataset 'cnn_dailymail_testset'. Dataset is accessible at https://smith.langchain.com/o/6691a6dd-a70e-56c0-8f45-a1f64338d797/datasets/920a9200-8c8c-46e5-a629-73bb91acaff9
